In [1]:
# Import dependencies
import gmaps
import requests
import pandas as pd

# import the API key
from config import g_key


In [2]:
# Store the CSV you saved created in part one into a DataFrame
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kavieng,-2.5744,150.7967,83.84,72,100,10.51,PG,2022-08-05 00:50:29
1,1,Port Augusta,-32.5000,137.7667,59.00,49,72,21.63,AU,2022-08-05 00:50:29
2,2,Kapaa,22.0752,-159.3190,84.52,68,0,10.36,US,2022-08-05 00:46:40
3,3,Bardiyah,31.7561,25.0865,76.75,72,0,9.19,LY,2022-08-05 00:50:30
4,4,Dikson,73.5069,80.5464,42.84,100,100,11.18,RU,2022-08-05 00:45:33


In [3]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])

In [7]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 75


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,Arraial Do Cabo,-22.9661,-42.0278,71.42,83,100,24.16,BR,2022-08-05 00:48:16
12,12,Rikitea,-23.1203,-134.9692,70.65,73,98,20.89,PF,2022-08-05 00:44:20
13,13,Nikolskoye,59.7035,30.7861,64.29,66,14,4.59,RU,2022-08-05 00:45:36
17,17,Muravlenko,63.7898,74.5230,64.27,60,94,8.72,RU,2022-08-05 00:50:33
18,18,Thompson,55.7435,-97.8558,64.56,45,75,3.44,CA,2022-08-05 00:42:17
22,22,Fortuna,40.5982,-124.1573,72.93,69,0,14.97,US,2022-08-05 00:50:35
26,26,Port Hedland,-20.3167,118.5667,68.05,37,0,16.11,AU,2022-08-05 00:50:36
28,28,Kapuskasing,49.4169,-82.4331,66.29,63,75,3.44,CA,2022-08-05 00:46:12
32,32,Rovaniemi,66.5000,25.7167,60.21,94,100,12.66,FI,2022-08-05 00:50:37
43,43,Doka,13.5167,35.7667,71.01,93,100,7.83,SD,2022-08-05 00:50:40


In [11]:
preferred_cities_df.count()

City_ID       193
City          193
Lat           193
Lng           193
Max Temp      193
Humidity      193
Cloudiness    193
Wind Speed    193
Country       192
Date          193
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Arraial Do Cabo,BR,71.42,-22.9661,-42.0278,
12,Rikitea,PF,70.65,-23.1203,-134.9692,
13,Nikolskoye,RU,64.29,59.7035,30.7861,
17,Muravlenko,RU,64.27,63.7898,74.5230,
18,Thompson,CA,64.56,55.7435,-97.8558,
22,Fortuna,US,72.93,40.5982,-124.1573,
26,Port Hedland,AU,68.05,-20.3167,118.5667,
28,Kapuskasing,CA,66.29,49.4169,-82.4331,
32,Rovaniemi,FI,60.21,66.5000,25.7167,
43,Doka,SD,71.01,13.5167,35.7667,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [16]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel Not Found... skipping")
        print(hotel_df)

Hotel Not Found... skipping
                City Country  Max Temp      Lat       Lng  \
7    Arraial Do Cabo      BR     71.42 -22.9661  -42.0278   
12           Rikitea      PF     70.65 -23.1203 -134.9692   
13        Nikolskoye      RU     64.29  59.7035   30.7861   
17        Muravlenko      RU     64.27  63.7898   74.5230   
18          Thompson      CA     64.56  55.7435  -97.8558   
..               ...     ...       ...      ...       ...   
547         Shizunai      JP     66.70  42.3339  142.3669   
550          Mirador      BR     74.97  -6.3708  -44.3631   
557            Macae      BR     69.48 -22.3708  -41.7869   
568         Tiassale      CI     69.46   5.8983   -4.8283   
573       Guiratinga      BR     72.59 -16.3494  -53.7583   

                                 Hotel Name  
7                       Pousada Porto Praia  
12                          People ThankYou  
13                  Tourist House - Sablino  
17                    Hostel Rus Muravlenko  
18       

Hotel Not Found... skipping
                City Country  Max Temp      Lat       Lng  \
7    Arraial Do Cabo      BR     71.42 -22.9661  -42.0278   
12           Rikitea      PF     70.65 -23.1203 -134.9692   
13        Nikolskoye      RU     64.29  59.7035   30.7861   
17        Muravlenko      RU     64.27  63.7898   74.5230   
18          Thompson      CA     64.56  55.7435  -97.8558   
..               ...     ...       ...      ...       ...   
547         Shizunai      JP     66.70  42.3339  142.3669   
550          Mirador      BR     74.97  -6.3708  -44.3631   
557            Macae      BR     69.48 -22.3708  -41.7869   
568         Tiassale      CI     69.46   5.8983   -4.8283   
573       Guiratinga      BR     72.59 -16.3494  -53.7583   

                                 Hotel Name  
7                       Pousada Porto Praia  
12                          People ThankYou  
13                  Tourist House - Sablino  
17                    Hostel Rus Muravlenko  
18       

Hotel Not Found... skipping
                City Country  Max Temp      Lat       Lng  \
7    Arraial Do Cabo      BR     71.42 -22.9661  -42.0278   
12           Rikitea      PF     70.65 -23.1203 -134.9692   
13        Nikolskoye      RU     64.29  59.7035   30.7861   
17        Muravlenko      RU     64.27  63.7898   74.5230   
18          Thompson      CA     64.56  55.7435  -97.8558   
..               ...     ...       ...      ...       ...   
547         Shizunai      JP     66.70  42.3339  142.3669   
550          Mirador      BR     74.97  -6.3708  -44.3631   
557            Macae      BR     69.48 -22.3708  -41.7869   
568         Tiassale      CI     69.46   5.8983   -4.8283   
573       Guiratinga      BR     72.59 -16.3494  -53.7583   

                                 Hotel Name  
7                       Pousada Porto Praia  
12                          People ThankYou  
13                  Tourist House - Sablino  
17                    Hostel Rus Muravlenko  
18       

In [17]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))